## Simple Pendulum

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
me.init_vprinting()

In [ ]:
theta, omega = me.dynamicsymbols("theta, omega")
thetad = me.dynamicsymbols("theta", 1)
L = symbols("L")

M, N, A = symbols("M, N, A", cls=me.ReferenceFrame)
N.orient_axis(M, M.z, pi)
A.orient_axis(N, N.z, theta)

O, P, G = symbols("O, P, G", cls=me.Point)
G.set_pos(O, L / 2 * A.y)
P.set_pos(O, L * A.y)

O.set_vel(N, 0)
G.v2pt_theory(O, N, A)
G.a2pt_theory(O, N, A)
P.v2pt_theory(O, N, A)
P.a2pt_theory(O, N, A)

In [ ]:
q_ind = [theta]
u_ind = [omega]
kde = [thetad - omega]

kane = me.KanesMethod(
    N,
    q_ind=q_ind,
    u_ind=u_ind,
    kd_eqs=kde
)

In [ ]:
m, rho, g = symbols("m, rho, g")
cord_mass = rho * L
cord = me.RigidBody(
    "Cord", G, A, cord_mass,
    (me.inertia(A, rho * L**3 / 12, 0, rho * L**3 / 12), G))
particle = me.Particle("Particle", P, m)

In [ ]:
bodies = [cord, particle]
loads = [
    (G, cord_mass * g * N.y),
    (P, m * g * N.y)
]

In [ ]:
fr, frstar = kane.kanes_equations(bodies, loads)
fr + frstar

In [ ]:
mm = kane.mass_matrix_full
fm = kane.forcing_full
eom = kane.rhs() # it performs mm.inv() * fm
t = symbols("t")
display(mm, fm, Eq(Matrix([thetad, omega.diff(t)]), eom))

In [ ]:
rhs = eom[1].subs(rho, 0)
Eq(thetad.diff(t), rhs)

### Adding friction and Reaction forces to the Pendulum.

In [ ]:
theta, omega = me.dynamicsymbols("theta, omega")
thetad = me.dynamicsymbols("theta", 1)
# reaction forces acting on point O
Rx, Ry, Rz = me.dynamicsymbols("R_x, R_y, R_z")
# auxiliary velocities needed to compute the reaction forces
u1_aux, u2_aux, u3_aux = me.dynamicsymbols("u_aux^((1:4))")

L, m, rho, g, d = symbols("L, m, rho, g, d")

M, N, A = symbols("M, N, A", cls=me.ReferenceFrame)
N.orient_axis(M, M.z, pi)
A.orient_axis(N, N.z, theta)

O, P, G = symbols("O, P, G", cls=me.Point)
G.set_pos(O, L / 2 * A.y)
P.set_pos(O, L * A.y)

# IMPORTANT: set fictitious generalized speeds on the point where
# the reaction forces are acting on.
O.set_vel(N, u1_aux * N.x + u2_aux * N.y + u3_aux * N.z)
G.v2pt_theory(O, N, A)
G.a2pt_theory(O, N, A)
P.v2pt_theory(O, N, A)
P.a2pt_theory(O, N, A)

q_ind = [theta]
u_ind = [omega]
u_aux = [u1_aux, u2_aux, u3_aux]
kde = [thetad - omega]

kane = me.KanesMethod(
    N,
    q_ind=q_ind,
    u_ind=u_ind,
    kd_eqs=kde,
    u_auxiliary=u_aux
)

cord_mass = rho * L
cord = me.RigidBody(
    "Cord", G, A, cord_mass,
    (me.inertia(A, rho * L**3 / 12, 0, rho * L**3 / 12), G))
particle = me.Particle("Particle", P, m)

bodies = [cord, particle]
loads = [
    (G, cord_mass * g * N.y),
    (P, m * g * N.y),
    # damping torque
    (A, -d * omega * A.z),
    (N, d * omega * A.z),
    # reaction forces acting on O
    (O, Rx * N.x + Ry * N.y + Rz * N.z)
]

fr, frstar = kane.kanes_equations(bodies, loads)
fr + frstar

In [ ]:
mm = kane.mass_matrix_full
fm = kane.forcing_full
(mm, fm, mm.inv() * fm)

In [ ]:
kane.auxiliary_eqs

In [ ]:
R_eqns = solve(kane.auxiliary_eqs, [Rx, Ry, Rz])
R_eqns = Matrix(list(R_eqns.values())).T
R_eqns.T

### Extraction of information with Scipy.

In [ ]:
from scipy.integrate import solve_ivp
import numpy as np

mm = kane.mass_matrix_full
fm = kane.forcing_full
constants = {L: 2, g: 9.81, m: 1, rho: 0, d: 1.25}

t = symbols("t")
arguments_1 = [theta, omega] + list(constants.keys())
arguments_2 = (
    [theta, omega] +
    [theta.diff(t), omega.diff(t)] +
    list(constants.keys())
)

mm_func = lambdify(arguments_1, mm, cse=True)
fm_func = lambdify(arguments_1, fm, cse=True)
R_eqns_func = lambdify(arguments_2, R_eqns, cse=False)

def func_to_integrate(t, x, args):
    values = np.concatenate((x, args))
    solution = np.linalg.solve(
        mm_func(*values),
        fm_func(*values)
    )
    return solution.flatten()

t_span = [0, 10]
x0 = [np.pi/6, 0]
fps = 60
t_eval = np.linspace(*t_span, int(fps * (t_span[1] - t_span[0])))
results = solve_ivp(func_to_integrate, t_span, x0, args=(list(constants.values()), ), t_eval=t_eval)
results

In [ ]:
import matplotlib.pyplot as plt

results_deg = np.rad2deg(results.y)
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(results.t, results_deg[0, :])
ax[0].set_ylabel(r"$\theta$ [deg]")
ax[1].plot(results.t, results_deg[1, :])
ax[1].set_ylabel(r"$\omega$ [deg / s]")
ax[1].set_xlabel("Time [s]")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots(figsize=(5, 2.5), layout='constrained')
ax2 = ax1.twinx()
ax1.plot(results.t, results_deg[0, :], color="k")
ax1.set_ylabel(r"$\theta$ [deg]")
ax1.set_xlabel("Time [s]")
ax2.plot(results.t, results_deg[1, :], linestyle="--", color="k")
ax2.set_ylabel(r"$\omega$ [deg / s]")
fig.legend([r"$\theta$", r"$\omega$"], ncol=2, loc="outside lower center")
fig.savefig("24-simple-pendulum-2.pgf")

In [ ]:
v_P = P.vel(N).to_matrix(N)
v_P

In [ ]:
kindiffdict = kane.kindiffdict()
aux_dict = {aux_speed: 0 for aux_speed in u_aux}
display(kindiffdict, aux_dict)

In [ ]:
v_P = v_P.subs(kindiffdict).subs(aux_dict)
v_P

In [ ]:
v_P_x_func = lambdify([L, theta, omega], v_P[0])
v_P_y_func = lambdify([L, theta, omega], v_P[1])
v_P_x_res = v_P_x_func(constants[L], results.y[0, :], results.y[1, :])
v_P_y_res = v_P_y_func(constants[L], results.y[0, :], results.y[1, :])
fig, ax = plt.subplots()
ax.plot(results.t, v_P_x_res, label=r"$\vec{v}_{P} \cdot \hat{n}_{x}$")
ax.plot(results.t, v_P_y_res, label=r"$\vec{v}_{P} \cdot \hat{n}_{y}$")
ax.legend()
ax.set_xlabel("Time [s]")
ax.set_ylabel("Velocity [m/s]")
plt.show()

In [ ]:
v_P_x_func = lambdify([L, theta, omega], v_P[0])
v_P_y_func = lambdify([L, theta, omega], v_P[1])
v_P_x_res = v_P_x_func(constants[L], results.y[0, :], results.y[1, :])
v_P_y_res = v_P_y_func(constants[L], results.y[0, :], results.y[1, :])
fig, ax = plt.subplots(figsize=(5, 2.5), layout='constrained')
ax.plot(results.t, v_P_x_res, "k", label=r"$\vec{v}_{P} \cdot \hat{n}_{x}$")
ax.plot(results.t, v_P_y_res, "k--", label=r"$\vec{v}_{P} \cdot \hat{n}_{y}$")
fig.legend(ncol=2, loc="outside lower center")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Velocity [m/s]")
plt.show()
fig.savefig("24-simple-pendulum-3.pgf")

In [ ]:
results_matrix = results.y.T
states_dot = np.zeros_like(results_matrix)
reactions = np.zeros((results_matrix.shape[0], 3))

for i in range(results_matrix.shape[0]):
    args_1 = list(results_matrix[i, :]) + list(constants.values())
    states_dot[i, :] = np.linalg.solve(
        mm_func(*args_1),
        fm_func(*args_1)
    ).flatten()
    args_2 = list(results_matrix[i, :]) + list(states_dot[i, :]) + list(constants.values())
    reactions[i, :] = R_eqns_func(*args_2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(results.t, reactions[:, 0], label="$R_{x}$")
ax.plot(results.t, reactions[:, 1], label="$R_{y}$")
ax.plot(results.t, reactions[:, 2], label="$R_{z}$")
ax.set_xlabel("time [s]")
ax.set_ylabel("force [N]")
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5), layout='constrained')
ax.plot(results.t, reactions[:, 0], "k", label="$R_{x}$")
ax.plot(results.t, reactions[:, 1], "k--", label="$R_{y}$")
ax.plot(results.t, reactions[:, 2], "k:", label="$R_{z}$")
fig.legend(ncol=3, loc="outside lower center")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Force [N]")
plt.show()
fig.savefig("24-simple-pendulum-4.pgf")

### Animation with Matplotlib

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots()
xp = constants[L] * np.sin(results.y[0, 0])
yp = -constants[L] * np.cos(results.y[0, 0])
string_handle, = ax.plot([0, xp], [0, yp], color="darkgray")
mass_handle = ax.scatter([xp], [yp], color="k")
ax.set_aspect("equal")
ax.axis([-2, 2, -2.5, 0.5])

def update(idx):
    xp = constants[L] * np.sin(results.y[0, idx])
    yp = -constants[L] * np.cos(results.y[0, idx])
    string_handle.set_data([0, xp], [0, yp])
    mass_handle.set_offsets([xp, yp])
    ax.set_title("Time = {:.2f} s".format(results.t[idx]))

ani = FuncAnimation(fig, update, frames=len(results.t))
HTML(ani.to_jshtml(fps=60))
# optionally, save it to disk
# ani.save("simple-pendulum.mp4", fps=fps)

### Extraction of information with PyDy.

In [ ]:
import numpy as np
from pydy.system import System

constants = {L: 2, g: 9.81, m: 1, rho: 0, d: 1.25}

sys = System(kane)
sys.constants = constants
sys.initial_conditions = {
    theta: np.pi/6,
    omega: 0
}

fps = 60
t0, tf = 0, 10
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)

results = sys.integrate()

In [ ]:
sys.integrate??

In [ ]:
import matplotlib.pyplot as plt

results_deg = np.rad2deg(results)
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(sys.times, results_deg[:, 0])
ax[0].set_ylabel(r"$\theta$ [deg]")
ax[1].plot(sys.times, results_deg[:, 1])
ax[1].set_ylabel(r"$\omega$ [deg / s]")
ax[1].set_xlabel("Time [s]")
plt.show()

### Animation with PyDy

In [ ]:
from pydy.viz import Scene, Cylinder, Sphere, VisualizationFrame

In [ ]:
help(VisualizationFrame)

In [ ]:
cord_geom = Cylinder(length=L, radius=0.05, color='grey', name="cord")
particle_geom = Sphere(0.2, color="black", name="particle")

cord_vf = VisualizationFrame('cord vf', A, G, cord_geom)
particle_vf = VisualizationFrame('particle vf', N, P, particle_geom)

# arguments to Scene: reference frame, origin, visual frames, system
scene = Scene(M, O, cord_vf, particle_vf, system=sys)
scene.display_jupyter(axes_arrow_length=1)

In [ ]:
phi = 0

# new reference frame for the cord
AA = me.ReferenceFrame("A")
# rotate the cord about NN.z by some value
AA.orient_axis(M, M.z, phi)

OO = me.Point("OO")
PP1 = OO.locatenew("PP1", L * AA.x)
GG1 = OO.locatenew("GG1", L / 2 * AA.x)
PP2 = OO.locatenew("PP2", L * AA.y)
GG2 = OO.locatenew("GG2", L / 2 * AA.y)

cylinder_x = Cylinder(
    length=L, radius=0.1, name="cylinder aligned with x", color="red")
cylinder_x_vf = VisualizationFrame('cylinder x vf', AA, GG1, cylinder_x)

cylinder_y = Cylinder(
    length=L, radius=0.1, name="cylinder aligned with y", color="green")
cylinder_y_vf = VisualizationFrame('cylinder y vf', AA, GG2, cylinder_y)

scene = Scene(M, OO, cylinder_x_vf, cylinder_y_vf, system=sys)
scene.display_jupyter(axes_arrow_length=0.5)

In [ ]:
phi = pi / 6

# new reference frame for the cord
AA = me.ReferenceFrame("A")
# rotate the cord about NN.z by some value
AA.orient_axis(M, M.z, phi)

OO = me.Point("OO")
PP1 = OO.locatenew("PP1", L * AA.x)
GG1 = OO.locatenew("GG1", L / 2 * AA.x)
PP2 = OO.locatenew("PP2", L * AA.y)
GG2 = OO.locatenew("GG2", L / 2 * AA.y)

cylinder_x = Cylinder(
    length=L, radius=0.1, name="cylinder aligned with x", color="red")
cylinder_x_frame = me.ReferenceFrame("cf")
cylinder_x_frame.orient_axis(AA, AA.z, -pi/2)
cylinder_x_vf = VisualizationFrame('cylinder x vf', cylinder_x_frame, GG1, cylinder_x)

cylinder_y = Cylinder(
    length=L, radius=0.1, name="cylinder aligned with y", color="green")
cylinder_y_vf = VisualizationFrame('cylinder y vf', AA, GG2, cylinder_y)

scene = Scene(M, OO, cylinder_x_vf, cylinder_y_vf, system=sys)
scene.display_jupyter(axes_arrow_length=0.5)

## Simple Pendulum wit Joints Framework

In [ ]:
theta, omega = me.dynamicsymbols("theta, omega")
m, rho, g, L, d = symbols("m, rho, g, L, d")
# reaction forces acting on point O
Rx, Ry, Rz = me.dynamicsymbols("R_x, R_y, R_z")
# auxiliary velocities needed to compute the reaction forces
u1_aux, u2_aux, u3_aux = me.dynamicsymbols("u_aux^((1:4))")

In [ ]:
M, N, A, B = symbols("M, N, A, B", cls=me.ReferenceFrame)
N.orient_axis(M, M.z, pi)
B.orient_axis(A, A.z, 0)

# O: origin
# P: particle
# G: center of mass of the cord
O, P, G = symbols("O, P, G", cls=me.Point)

In [ ]:
# IMPORTANT: set fictitious generalized speeds on the point where
# the reaction forces are acting on.
O.set_vel(N, u1_aux * N.x + u2_aux * N.y + u3_aux * N.z)
O.vel(N)

In [ ]:
wall = me.RigidBody("Wall", O, N, 0)
# inertia wrt the cord's center of mass
cord_inertia = me.inertia(A, rho * L**3 / 12, 0, rho * L**3 / 12)
cord = me.RigidBody("Cord", G, A, rho * L, (cord_inertia, G))
particle = me.Particle("Particle", P, m)

In [ ]:
# system = me.System.from_newtonian(wall)
system = me.System(N, O) # equivalent to the previous line of code
system.add_auxiliary_speeds(u1_aux, u2_aux, u3_aux)

In [ ]:
joint1 = me.PinJoint(
    "J1", wall, cord, coordinates=theta, speeds=omega,
    child_point=-L/2 * cord.frame.y,
    joint_axis=wall.frame.z
)

In [ ]:
joint2 = me.WeldJoint(
    "J2", cord, particle,
    parent_point=L/2 * cord.frame.y,
    parent_interframe=B
)

In [ ]:
display((G.pos_from(O), P.pos_from(G), P.pos_from(O)))

In [ ]:
system.add_joints(
    joint1, joint2
)

In [ ]:
system.joints

In [ ]:
system.bodies

In [ ]:
system.kdes

In [ ]:
O.vel(N)

In [ ]:
O.set_vel(N, u1_aux * N.x + u2_aux * N.y + u3_aux * N.z)
G.v2pt_theory(O, N, A)
P.v2pt_theory(O, N, A)

In [ ]:
system.apply_uniform_gravity(g * system.y)
system.add_loads(
    # damping torque
    me.Torque(A, -d * omega * A.z),
    me.Torque(N, d * omega * A.z),
    # reaction forces acting on O
    me.Force(O, Rx * N.x + Ry * N.y + Rz * N.z)
)

In [ ]:
system.validate_system()
eoms = system.form_eoms()
eoms

In [ ]:
system.mass_matrix_full, system.forcing_full

In [ ]:
system.eom_method.auxiliary_eqs